Only reading in one email for now

In [58]:
enroncsv = "../experiments/data/anon.csv"
metadataHeaders = '../experiments/data/metadataHeaders.csv'
spam = '../experiments/data/spam.csv'
spamSubjects = '../experiments/data/spamWords.csv'

In [59]:
import pandas as pd

columns = pd.read_csv(metadataHeaders, sep=',').columns.tolist()

In [60]:
df = pd.read_csv(enroncsv, names=columns, sep='|', low_memory=False)

print("-- DONE --")

-- DONE --


In [61]:
removableColumns = pd.read_csv('data/removableColumns.csv', sep=',').columns.tolist()

df.drop(removableColumns, axis=1)

print('-- DONE --')

-- DONE --


In [5]:
listOfEmailsForBagOfWords = df.loc[df['Directory'].str.contains('inbox')]

print('-- DONE --')

-- DONE --


In [6]:
wordsPerEmail = df[pd.notnull(df['Subject'])]
nonSpamSubjects = wordsPerEmail['Subject']
wordsPerEmail = nonSpamSubjects.str.split(' ')
len(wordsPerEmail)

498161

In [7]:
import string
printable = set(string.printable)

def isEnglish(s):
    for x in s:
        if x not in printable:
            return False
    return True

def addToBagOfWords(dictionary, arr):
    for word in arr:
        word = word.lower()
        
        if '=' in word:
            continue
        if '/' in word:
            continue
        if '\\' in word:
            continue
        if '_' in word:
            continue
        if '-' in word:
            continue
        if ':' in word:
            continue
        if '@' in word:
            continue
        if '#' in word:
            continue
        if '$' in word:
            word = '$'

        word = word.replace('.', '')
        word = word.replace(')', '')
        word = word.replace('(', '')
        word = word.replace('&', '')
        word = word.replace('\'', '')
        word = word.replace('\"', '')
        word = word.replace(',', '')
        word = word.replace('[', '')
        word = word.replace(']', '')
        word = word.replace('{', '')
        word = word.replace('}', '')
        word = word.replace(';', '')

        if word == '':
            continue

        if word in dictionary:
            dictionary[word] = dictionary[word] + 1
        else:
            dictionary[word] = 1
            
    return dictionary

In [8]:
def addToDictionary(dictionary, arr):
    for index, subject in arr.iteritems():
        dictionary = addToBagOfWords(dictionary, subject)
                
    return dictionary

In [9]:
dictionary = {}

dictionary = addToDictionary(dictionary, wordsPerEmail)
        
len(dictionary)

39519

In [10]:
spamSubjects = pd.read_csv(spam, names=['Subject'], sep=',')

In [11]:
wordsPerEmail = spamSubjects['Subject'].str.split(' ')

dictionary = addToDictionary(dictionary, wordsPerEmail)
        
len(dictionary)

45090

In [12]:
dictionaryList = []
for key in dictionary:
    dictionaryList.append([key, dictionary[key]])
    
dictionaryList = pd.DataFrame(dictionaryList, columns=["Word", "Occurances"])
len(dictionaryList)

45090

We will only use words that occurred more than 30 times in our bag of words as other words would almost never match up anyway

In [13]:
dictionaryList = dictionaryList.loc[dictionaryList['Occurances'] > 30].reset_index(drop=True)
len(dictionaryList)

5751

In [14]:
def addBagToInputOutput(inputItems, outputItems, arr):
    for index, subject in inputItems.iteritems():
        posBagOfWords = {}
        posBagOfWords = addToBagOfWords(posBagOfWords, subject)

        listOfIn = []
        for key in posBagOfWords:
            listOfIn.append(key)

        inputBag = dictionaryList['Word'].isin(listOfIn).values.tolist()

        arr.append([np.array(inputBag), np.array(outputItems)])
        
    return arr
    

### Create training data

In [15]:
import matplotlib.pyplot as plt
import numpy as np

training = []

NON_SPAM_SAMPLES = 1000
SPAM_SAMPLES = 1000

trainingPos = df[pd.notnull(df['Subject'])]
trainingPos = trainingPos.sample(NON_SPAM_SAMPLES)
trainingPos = trainingPos['Subject']

trainingPosSplit = trainingPos.str.split(' ')

training = addBagToInputOutput(trainingPosSplit, [0], training)
    
trainingNeg = spamSubjects.sample(SPAM_SAMPLES)
trainingNeg = trainingNeg['Subject']

trainingNegSplit = trainingNeg.str.split(' ')
    
training = addBagToInputOutput(trainingNegSplit, [1], training)

training = np.array(training)
np.random.shuffle(training)

len(training)

2000

Lets set up a neuarl network with neurons on each layer 

    input - 5558 + bias
    hidden layer 1 - 200 + bias
    hidden layer 2 - 20 + bias
    output - 1
    
The learing rate is set to 0.001
The netword is bound to (-2, 2)

In [16]:
from mlpy.numberGenerator.bounds import Bounds
from mlpy.neuralNetwork.feedForwardNeuralNetwork import NeuralNetwork
from mlpy.neuralNetwork.structure.layer import Layer

l_rate = 0.001
bounds = Bounds(-2, 2)

inputLayer = Layer(bounds, size = len(training[0][0]), prev = None, l_rate = l_rate, bias = True, label = "Input layer")
hiddenLayer = Layer(bounds, size = 200, prev = inputLayer, l_rate = l_rate, bias = True, label = "Hidden layer")
hiddenLayer2 = Layer(bounds, size = 20, prev = hiddenLayer, l_rate = l_rate, bias = True, label = "Hidden layer 2")
outputLayer = Layer(bounds, size = len(training[0][1]), prev = hiddenLayer2, l_rate = l_rate, bias = False, label = "Output layer")

fnn = NeuralNetwork()
fnn.appendLayer(inputLayer)
fnn.appendLayer(hiddenLayer)
fnn.appendLayer(hiddenLayer2)
fnn.appendLayer(outputLayer)

group_training = np.array([input[0] for input in training])
group_target = np.array([output[1] for output in training])

errors = []

We will run training over 4000 iterations and output the mean error every 200 iterations.

In [17]:
ITERATIONS = 4000

print("Starting..")
for i in range(ITERATIONS):
    mod = i % len(training)
    in_out = training[mod]
    result = fnn.fire(group_training)
    error = fnn.backPropagation(group_target)
    
    if i % 200 == 0:
        print(str(np.round(i/ITERATIONS*100)) + '%\t', error.mean())
    
print("-- DONE --")

Starting..
0.0%	 -0.2765026234057767
5.0%	 -0.003050694372103209
10.0%	 0.00010212984489931553
15.0%	 -7.728464215698416e-05
20.0%	 -0.0008527255661347808
25.0%	 -0.0016278042466524954
30.0%	 -0.0021047060910248804
35.0%	 -0.0023617466117687434
40.0%	 -0.0022234871782112735
45.0%	 -0.0020463127504577773
50.0%	 -0.0015229964999428193
55.0%	 -0.0010783260052427495
60.0%	 -0.0007169712542621996
65.0%	 -0.0003052411620596356
70.0%	 -0.0002725217854694817
75.0%	 -0.0003113240461118918
80.0%	 -0.0003045537326310288
85.0%	 9.88297671506404e-05
90.0%	 -0.00010332489543996725
95.0%	 0.00012965235437434442
-- DONE --


### Create testing data

In [18]:
testing = []

TEST_NON_SPAM_SAMPLES = 250
TEST_SPAM_SAMPLES = 250

trainingPos = df[pd.notnull(df['Subject'])]
trainingPos = trainingPos.sample(TEST_NON_SPAM_SAMPLES)
trainingPos = trainingPos['Subject']

trainingPosSplit = trainingPos.str.split(' ')

testing = addBagToInputOutput(trainingPosSplit, [0], testing)
    
trainingNeg = spamSubjects.sample(TEST_SPAM_SAMPLES)
trainingNeg = trainingNeg['Subject']

trainingNegSplit = trainingNeg.str.split(' ')

testing = addBagToInputOutput(trainingNegSplit, [1], testing)

testing = np.array(testing)
np.random.shuffle(testing)

len(testing)

500

Testing the model we will take the test data and use it as an measure of performace. The following will be output:

1. The Classification Accuracy
2. The Non-spam Classsification accuracy
3. The Spam Classification accuracy

In [19]:
correct = 0
spamCorrect = 0
nonSpamCorrect = 0
for i in range(len(testing)):
    in_out = testing[i]
    result = fnn.fire(np.array([in_out[0]]))
    
    target = in_out[1][0]
    result = np.round(result[0][0])

    if result == target:
        correct += 1
        if target == 1:
            spamCorrect += 1
        else:
            nonSpamCorrect += 1
        
        
print("Classification accuracy: ", correct / len(testing))
print("Non spam classification accuracy: ", nonSpamCorrect / TEST_NON_SPAM_SAMPLES)
print("Spam classification accuracy: ", spamCorrect / TEST_SPAM_SAMPLES)

Classification accuracy:  0.754
Non spam classification accuracy:  0.744
Spam classification accuracy:  0.764


### Lets apply the classifier to a subset of the data to see what it classifies as spam

In [77]:
dfClassify = df.drop(['Filename', 'Person', 'Directory', 'Message-ID', 'Content-Transfer-Encoding', 'Content-Type', 'Date', 'X-FileName', 'From', 'X-To', 'X-bcc', 'Cc', 'X-cc', 'To', 'X-Folder', 'X-Origin', 'Time', 'Bcc', 'X-From', 'Attendees', 'Re', 'Mime-Version'], axis=1)

dfClassify = dfClassify[pd.notnull(df['Subject'])]
dfClassify = dfClassify.sample(100)

def classify(arr):
    classifySplit = arr.split(' ')
    
    posBagOfWords = {}
    posBagOfWords = addToBagOfWords(posBagOfWords, classifySplit)

    listOfIn = []
    for key in posBagOfWords:
        listOfIn.append(key)

    inputBag = dictionaryList['Word'].isin(listOfIn).values.tolist()
    
    result = fnn.fire(np.array([np.array(inputBag)]))[0][0]
    
    return result

dfClassify['Classification'] = dfClassify['Subject'].apply(classify)
    
dfClassify[dfClassify['Classification'] > 0.9]

,Subject,Classification
115264,Deal 156071 Feb 00,0.916513
131332,Sivy on Stocks: Winning a loser's game,0.995735
98391,RE: can't you defer the ecogas depos? richard ...,0.999483
74051,4/4/01 Churn,0.927261
360290,FW: Exchange Cash,0.915055
300667,RE: CLE Program - Antitrust Issues Relating to...,0.976513
300448,RE: phone numbers,0.971287
493360,Australia & Japan,0.974225
100269,old bills,0.998415
